Working with Elise to speed up the matching of true halos to observed mock Gaia stars. 

In [1]:
import pandas as pd 
import numpy as np
import h5py 
from itertools import izip
import cPickle as pickle 
from collections import OrderedDict

In [2]:
with open('parentid_dicts.pkl', 'r') as f:
    parentid_dicts = pickle.load(f)

In [9]:
mask_dict = OrderedDict()
#i = 5
for i in xrange(5, 205): #, 600):
    print i,
    fname = '/oak/stanford/orgs/kipac/users/enadler/m12i_rockstar_dm/catalog_hdf5/star_%03d.hdf5'%i

    with h5py.File(fname, 'r') as f:
        mask = np.array(f['star.number']) != -1
        ar = np.array(f['star.indices'])[mask]
        halo_ID = np.array(f['id'])[mask]

        for k, stars in enumerate(ar):
            key = halo_ID[k]
            mask_dict[key] = []
            for j in xrange(10):
                mask1 = [parentid_dicts[j][s] for s in stars if s in parentid_dicts[j]]
                mask_dict[key].append(mask1)

 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204


In [10]:
print len(mask_dict)

17376


In [11]:
keys= ['ra_true', 'dec_true', 'pmra_true', 'pmdec_true', 'parallax_true', 'b_true']

In [12]:
mean_vals = np.zeros((len(mask_dict.keys()), len(keys)))

In [ ]:
for j in range(10):
    fname1 = '/oak/stanford/orgs/kipac/edarragh/m12i_lsr0/slice-%d.hdf5'%j
    print '*'*10
    if not any([v[j] for v in mask_dict.itervalues()]):
        continue # don't load this file if there's nothing in them 
    with h5py.File(fname1, 'r') as f1:
        for k_no, k in enumerate(keys):
            print '~'*10
            val = f1[k][()] # this is a slow disk read
            
            for obj_no, (obj_key, obj_mask) in enumerate(mask_dict.iteritems()):
                print j, k_no, obj_no
                if len(obj_mask)==0 or len(obj_mask[j]) == 0:
                    continue
                #print j, k_no, obj_no
                mean_vals[obj_no, k_no] = val[obj_mask[j]].mean()
            del val

In [ ]:
np.save('mean_vals.npy', mean_vals)